# README
## Options:
### Pretrained model:
- word2vec-ruscorpora-300	
- glove-twitter-200

### Tokenization: How to manage OOV?
- Link: '&'
- Tag: '#'
- Mention: '@' 

In [4]:
import copy
import pandas as pd
import numpy as np
import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import SnowballStemmer

import gensim
import gensim.models.word2vec as w2v
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score
from transformers import AdamW, get_linear_schedule_with_warmup

from fetchData import fetchdata, cv_events
import __MLP
import __Preprocessing
# from __MLP import getSamplers, convert_df_to_unsqueezed_tensor, train_sequential, clf_report
import random



# from __Preprocess import *

In [2]:
pd.set_option('display.max_rows', 400)
# pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 400)

In [3]:
import gensim.downloader as api
model = api.load('glove-twitter-200')

# ! Functions

In [5]:
""" Replaces contractions from a string to their equivalents """
def replaceContraction(text):
    contraction_patterns = [ (r'won\'t', 'will not'), (r'can\'t', 'cannot'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
                            (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not') ]
    patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
    for (pattern, repl) in patterns:
        (text, count) = re.subn(pattern, repl, text)
    return text

def getTokenization(raw_data):

    lmt = WordNetLemmatizer()
    stemmer = SnowballStemmer("english")
    freqdist = nltk.FreqDist()
    tweet_tokenizer = TweetTokenizer()
    tweet_tokens = []
    stop_words = set(stopwords.words('english'))

    for sent in raw_data.text:

        # 기존
        sent = replaceContraction(sent)
        sent = re.sub(r"(www\S+)|(http\S+)", "HTTPURL", sent)
        sent = re.sub(r"[A-Za-z0-9]+", "@USE", sent)
        sent = re.sub(r"(#)(\S+)", r'\1 \2', sent)
        sent = re.sub(r'([^\s\w@#&]|_)+', '', sent)

        # sent = re.sub('@[^\s]+','atUser',sent)
        # sent = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',sent)

        # sent = re.sub('', '', sent.lower())
        # sent = [tweet_tokenizer.tokenize(sent)]
        sent = tweet_tokenizer.tokenize(sent.lower())
        # sent = [stemmer.stem(token) for token in sent]
        # sent = [lmt.lemmatize(token) for token in sent]

        temp = [token for token in sent if not token in stop_words]
        tweet_tokens.append([temp])
        # tweet_tokens.append(tweet_tokenizer.tokenize(sent))
    df_tokens = pd.DataFrame(tweet_tokens, columns=['token'])
    return df_tokens

def getTokenization_less(raw_data):

    lmt = WordNetLemmatizer()
    stemmer = SnowballStemmer("english")
    freqdist = nltk.FreqDist()
    tweet_tokenizer = TweetTokenizer()
    tweet_tokens = []
    stop_words = set(stopwords.words('english'))

    for sent in raw_data.text:

        # 기존
        sent = re.sub(r"(www\S+)|(http\S+)", "*", sent)
        sent = re.sub(r"@\S+", "@", sent)
        sent = re.sub(r"(#)(\S+)", r'\1 \2', sent)
        sent = re.sub(r'([^\s\w@#&]|_)+', '', sent)

        # sent = re.sub('@[^\s]+','atUser',sent)
        # sent = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',sent)

        # sent = re.sub('', '', sent.lower())
        # sent = [tweet_tokenizer.tokenize(sent)]
        sent = replaceContraction(sent)
        sent = tweet_tokenizer.tokenize(sent.lower())
        # sent = [stemmer.stem(token) for token in sent]
        # sent = [lmt.lemmatize(token) for token in sent]

        temp = [token for token in sent if not token in stop_words]
        tweet_tokens.append([temp])
        # tweet_tokens.append(tweet_tokenizer.tokenize(sent))
    df_tokens = pd.DataFrame(tweet_tokens, columns=['token'])
    return df_tokens

def getObjectW2V(model):
    w2v_object = model.wv
    w2v_vectors = w2v_object.vectors # here you load vectors for each word in your model
    w2v_indices = {word: w2v_object.vocab[word].index for word in w2v_object.vocab} # here you load indices - with whom you can find an index of the particular word in your model 
    return w2v_object, w2v_vectors, w2v_indices

def get_W2V_AVG(df_tokens):
    import copy
    df_tokens['token_vec'] = copy.deepcopy(df_tokens['token'])
    

    for index, sent in enumerate(df_tokens['token_vec']):
        df_tokens['token_vec'][index] = vectorize(sent).mean(axis=0)

    df_temp = pd.DataFrame(
        df_tokens['token_vec'].values.tolist()).add_prefix('vec_avg')

    # df_tokens = df_tokens.join(df_temp).drop('token_vec', axis=1)
    df_temp = pd.DataFrame(df_tokens['token_vec'].values.tolist()).add_prefix('vec_avg')
    df_tokens = df_tokens.join(df_temp).drop('token_vec', axis=1)
    # df_test.drop('text_token_vec',axis=1, inplace=True)

    return df_tokens
    # return pd.DataFrame(df_tokens)

def vectorize(line):
    words = []
    for word in line:  # line - iterable, for example list of tokens
        try:
            w2v_idx = w2v_indices[word]
        except KeyError:  # if you does not have a vector for this word in your w2v model, continue
            words.append(list(np.zeros(200,)))
            continue
        words.append(list(w2v_vectors[w2v_idx]))
        if not word:
            words.append(None)

        if len(line) > len(words):
            continue
    return np.asarray(words)

# ! COVERSION: PHEME | PHEMEext | RHI

In [8]:
# raw_RHI = pd.read_csv("./data/_RHI_text.csv")
raw_PHEME = pd.read_csv("./data/_PHEME_text.csv")
raw_PHEMEext = pd.read_csv("./data/_PHEMEext_text.csv")

datasets = [raw_PHEME, raw_PHEMEext]

# for i, dataset in enumerate(datasets): datasets[i] = getTokenization(dataset)
datasets = [getTokenization_less(dataset) for dataset in datasets]
# datasets = [get_W2V_AVG(dataset) for dataset in datasets]
# w2v_object, w2v_vectors, w2v_indices = getObjectW2V(model)

In [9]:
datasets[1]

,token
0,"[micheal, essien, denying, ebola, rumours, like]"
1,"[truth, internet, rumours, contracted, ebolai, well, andamp, im, gud, andamp, training, usual, tomorrow, #, falsenews]"
2,"[essien, lawyers, considering, file, lawsuit, nigerian, media, reported, fake, ebola, story]"
3,"[good, news, rumours, michael, essien, contracted, ebola, virus, false]"
4,"[milan, stated, reports, essien, ebola, completely, false, @]"
...,...
618,"[franz, marc, horses, update, #, gurlitt, nazitainted, #, art, trove, via, @]"
619,"[munich, district, court, confirmed, application, cornelius, gurlitts, cousin, certificate, inheritance, details, today]"
620,"[gurlitt, collection, go, many, people, sounding, upcoming, press, conference, monday]"
621,"[possible, nazi, art, transfer, riles, jewish, groups, course, transfer, gurlitt, trove, swiss, museum]"


In [40]:
for i in raw_RHI.text:
    if (i.find("&") == True):
        print(i)

T&S+5 RT @CharlesMBlow AlaskaReport: "Todd and Sarah Palin to divorce: Affairs on both sides" Can this be true? http://tinyurl.com/nt6gfs


In [ ]:
datasets = [get_W2V_AVG(dataset) for dataset in datasets]

In [7]:
temp = get_W2V_AVG(datasets[0])

In [14]:
datasets[0].to_csv('./data/_RHI_text_AVGw2v.csv', index = False)
datasets[1].to_csv('./data/_PHEME_text_AVGw2v.csv', index = False)
datasets[2].to_csv('./data/_PHEMEext_text_AVGw2v.csv', index = False)

# 알아보기

In [16]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
# stop_words

In [158]:
raw_data = pd.read_csv("./data/_PHEME_text.csv")
raw_RHI = pd.read_csv("./data/_RHI_text.csv")

lmt = WordNetLemmatizer()
stemmer = SnowballStemmer("english")
freqdist = nltk.FreqDist()
tweet_tokenizer = TweetTokenizer()
tweet_tokens = []
stop_words = set(stopwords.words('english'))

""" Replaces contractions from a string to their equivalents """
contraction_patterns = [ (r'won\'t', 'will not'), (r'can\'t', 'cannot'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
                         (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not') ]
def replaceContraction(text):
    patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
    for (pattern, repl) in patterns:
        (text, count) = re.subn(pattern, repl, text)
    return text

for sent in raw_data.text:

    sent = re.sub(r"http\S+", "&", sent)
    # sent = re.sub(r"@\S+", "@", sent)
    sent = re.sub(r"(#)(\S+)", r'\1 \2', sent)

    sent = re.sub(r'([^\s\w@#&]|_)+', '', sent)
    sent = re.sub('@[^\s]+','atUser',sent)
    # sent = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',sent)
    # sent = re.sub(r'#([^\s]+)', r'\1', sent)

    sent = replaceContraction(sent)

    # sent = re.sub('', '', sent.lower())
    # print(tweet_tokenizer.tokenize(sent))
    # sent = [tweet_tokenizer.tokenize(sent)]
    sent = tweet_tokenizer.tokenize(sent.lower())
    sent = [stemmer.stem(token) for token in sent]
    # sent = [lmt.lemmatize(token) for token in sent]

    temp = [token for token in sent if not token in stop_words]
    tweet_tokens.append([temp])
    # tweet_tokens.append(tweet_tokenizer.tokenize(sent))
df_tokens = pd.DataFrame(tweet_tokens, columns=['token'])

## Import data

In [219]:
raw_data = pd.read_csv("./data/_PHEME_text.csv")

In [9]:
print(raw_data.shape)
raw_data.sample(5)

(5802, 2)


,text,Event
1080,"#ICYMI, Christopher Hitchens on the case for m...",charliehebdo
4120,DEVELOPING NEWS: Soldier shot at War Memorial....,ottawashooting
5004,RT @tomsteinfort: Terrifying photo of hostages...,sydneysiege
2305,#Ferguson chief said the officer was unaware o...,ferguson
4898,BREAKING: 2 people have run out of Sydney buil...,sydneysiege


## Preprocessing

## Tokenization

In [243]:
tweet_tokenizer = TweetTokenizer()
tweet_tokens = []

for sent in raw_data.text:
    sent = re.sub(r"http\S+", "&", sent)
    sent = re.sub(r"@\S+", "@", sent)
    sent = re.sub(r"#\S+", "#", sent)
    sent = re.sub(r'([^\s\w@#&]|_)+','', sent)
    sent = re.sub('','', sent.lower())
    # print(tweet_tokenizer.tokenize(sent))
    sent = [tweet_tokenizer.tokenize(sent)]
    # sent = [tweet_tokenizer.tokenize(sent.lower())]
    tweet_tokens.append(sent)
    # tweet_tokens.append(tweet_tokenizer.tokenize(sent))
df_tokens = pd.DataFrame(tweet_tokens, columns=['token'])


In [237]:
df_tokens.head()

,token
0,"[breaking, armed, man, takes, hostage, in, kosher, grocery, east, of, paris, &]"
1,"[#, killers, dead, confirmed, by, gendarmerie]"
2,"[top, french, cartoonists, charb, cabu, wolinski, tignous, confirmed, among, dead, in, #, #, attack, editor, is, critically, wounded]"
3,"[police, have, surrounded, the, area, where, the, #, attack, suspects, are, believed, to, be, &, &]"
4,"[photo, armed, gunmen, face, police, officers, near, #, hq, in, paris, &, &]"


## Word2Vec

### Downloading Pretrained model for Gensim

### Fetching pretrained Model and Convert the Raw Text

In [230]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
# corpus = api.load('text8')
# wv = api.load('word2vec-google-news-300')
# fasttext-wiki-news-subwords-300'
#  'glove-twitter-200',
model = api.load('glove-twitter-200')

In [20]:
# import inspect
# print(inspect.getsource(wv.__class__))

In [42]:
w2v_object = model.wv
w2v_vectors = w2v_object.vectors # here you load vectors for each word in your model
w2v_indices = {word: w2v_object.vocab[word].index for word in w2v_object.vocab} # here you load indices - with whom you can find an index of the particular word in your model 

In [241]:
def vectorize(line): 
    words = []
    for word in line: # line - iterable, for example list of tokens 
        try:
            w2v_idx = w2v_indices[word]
        except KeyError: # if you does not have a vector for this word in your w2v model, continue 
            words.append(list(np.zeros(200,)))
            continue
        words.append(list(w2v_vectors[w2v_idx]))
        if not word:
            words.append(None)

        if len(line) > len(words):
            continue
    return np.asarray(words)

In [233]:
# print("Tweet 1: ", raw_data['text'][1])
# print("Indice of '{}': {}".format(df_tokens['token'][1][0], w2v_indices[df_tokens['token'][1][0]]))
# # print("Indice of '{}': {}".format(raw_data['text_token'][1][0], w2v_vectors[w2v_indices[raw_data['text_token'][1][0]]]))
# # print("Indice of '{}': {}".format(raw_data['text_token'][1][1], w2v_indices[raw_data['text_token'][1][1]]))
# # print("Indice of '{}': {}".format(raw_data['text_token'][1][1], w2v_vectors[w2v_indices[raw_data['text_token'][1][1]]]))
# # print("\nVector of the first headline:\n", vectorize(raw_data['text_token'][1]))

In [244]:
import copy
df_tokens['token_vec'] = copy.deepcopy(df_tokens['token'])

for index, sent in enumerate(df_tokens['token_vec']):
    df_tokens['token_vec'][index] = vectorize(sent).mean(axis=0)

# df_test[['text_token','text_token_vec']].head()

df_temp = pd.DataFrame(df_tokens['token_vec'].values.tolist()).add_prefix('vec_avg')

df_tokens = df_tokens.join(df_temp).drop('token_vec',axis=1)
# df_test.drop('text_token_vec',axis=1, inplace=True)

In [1]:
df_tokens.head()

NameError: name 'df_tokens' is not defined